In [ ]:
# Install required dependencies
# Note: torch/torchvision are environment-specific; install manually if missing.
%pip install -q --upgrade pip
%pip install -q statsmodels scikit-image seaborn matplotlib pandas scipy opencv-python-headless

# Segment Anything
try:
    import segment_anything  # noqa: F401
except Exception:
    %pip install -q git+https://github.com/facebookresearch/segment-anything.git

# TIAToolbox (official HoVer-Net)
try:
    import tiatoolbox  # noqa: F401
except Exception:
    %pip install -q tiatoolbox

# TorchVision (used by lightweight HoverNet fallback)
try:
    import torchvision  # noqa: F401
except Exception:
    # If this fails, install torch/torchvision per your platform instructions
    %pip install -q torchvision

print("✅ Dependencies installation cell executed.")


In [ ]:
# Dependency installation / verification (runs in Python, no magics)
import sys
import subprocess
import importlib

def ensure_pkg(module_name: str, pip_spec: str | None = None) -> None:
    try:
        importlib.import_module(module_name)
        print(f"{module_name} OK")
    except Exception:
        spec = pip_spec or module_name
        print(f"Installing {spec} ...")
        subprocess.check_call([sys.executable, "-m", "pip", "install", "-q", spec])
        importlib.import_module(module_name)
        print(f"{module_name} OK")

# Core scientific & plotting
for mod, spec in [
    ("statsmodels", "statsmodels"),
    ("skimage", "scikit-image"),
    ("seaborn", "seaborn"),
    ("matplotlib", "matplotlib"),
    ("pandas", "pandas"),
    ("scipy", "scipy"),
    ("PIL", "Pillow"),
    ("cv2", "opencv-python-headless"),
]:
    ensure_pkg(mod, spec)

# Segment Anything
try:
    importlib.import_module("segment_anything")
    print("segment_anything OK")
except Exception:
    print("Installing Segment Anything from GitHub ...")
    subprocess.check_call([sys.executable, "-m", "pip", "install", "-q", "git+https://github.com/facebookresearch/segment-anything.git"])
    importlib.import_module("segment_anything")
    print("segment_anything OK")

# TIAToolbox (official HoVer-Net)
ensure_pkg("tiatoolbox", "tiatoolbox")

# TorchVision (fallback HoverNet uses it; attempt install if missing)
try:
    importlib.import_module("torchvision")
    print("torchvision OK")
except Exception:
    try:
        print("Installing torchvision ... (ensure it matches your torch version)")
        subprocess.check_call([sys.executable, "-m", "pip", "install", "-q", "torchvision"])
        importlib.import_module("torchvision")
        print("torchvision OK")
    except Exception:
        print("Warning: torchvision install failed; if using TIAToolbox HoVer-Net this is optional.")

# Torch presence + CUDA check
try:
    import torch  # noqa: F401
    import torch as _torch
    print("torch OK, CUDA:", _torch.cuda.is_available())
except Exception:
    print("Warning: torch not available. Install a platform-appropriate torch manually if needed.")

print("✅ Dependency check complete.")


# RQ1: SAM Variants vs Established Models on PanNuke

**Research Question**: Do different variants of the Segment Anything Model (SAM), including domain-adapted PathoSAM, achieve competitive or superior nuclei instance segmentation performance on PanNuke compared to established models (HoVer-Net, CellViT, LKCell) and a U-Net baseline?

- **H0 (Null)**: SAM variants do not significantly outperform established models in mPQ or detection F1.
- **H1 (Alt.)**: At least one SAM variant significantly outperforms baselines in mPQ or detection F1.

### What this notebook does
- Loads PanNuke tiles via a reusable dataset
- Runs inference for available models:
  - SAM variants (if checkpoints available)
  - U-Net baseline (checkpoint-gated)
- Converts predictions to instance masks and computes: PQ, object F1, AJI, Dice
- Performs paired statistics with multiple-comparison correction
- Saves CSVs, figures, and an HTML report under `reports/rq1`

Note: HoVer-Net, CellViT, and LKCell slots are scaffolded for future integration; this notebook focuses on SAM variants and a U-Net baseline to establish a robust, reproducible evaluation pipeline.


In [ ]:
import os
from pathlib import Path
import json
import random
from typing import Dict, List, Tuple

import numpy as np
import pandas as pd
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt
import seaborn as sns

# Project-local imports
import sys
if "__file__" in globals():
    SRC_DIR = Path(__file__).resolve().parent
else:
    SRC_DIR = Path.cwd()
sys.path.append(str(SRC_DIR))
from datasets.pannuke_tissue_dataset import PanNukeTissueDataset
from models.unet import UNet

# Metrics (instance-aware)
from metrics.seg_metrics import (
    reconstruct_instances,
    dice_coefficient,
    aji_aggregated_jaccard,
    pq_panoptic,
    f1_object,
)

# Reproducibility & device
seed = 42
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)

device = torch.device('cuda' if torch.cuda.is_available() else 'mps' if torch.backends.mps.is_available() else 'cpu')
print("Device:", device)

# Paths
PROJECT_ROOT = SRC_DIR.parent
DATASET_TISSUES = PROJECT_ROOT / "dataset_tissues"
REPORTS_DIR = PROJECT_ROOT / "reports" / "rq1"
FIG_DIR = REPORTS_DIR / "figures"
CSV_DIR = REPORTS_DIR / "tables"
for d in [REPORTS_DIR, FIG_DIR, CSV_DIR]:
    d.mkdir(parents=True, exist_ok=True)

sns.set_style("whitegrid")
sns.set_context("notebook")


In [ ]:
# Dataset setup
available_tissues = [p.name for p in DATASET_TISSUES.iterdir() if p.is_dir()]
print("Tissues:", len(available_tissues))

IMG_SIZE = 256
BATCH_SIZE = 6

# Simple transforms via dataset defaults; they already resize/normalize if needed

def make_loader(tissue: str, split: str = "test") -> DataLoader:
    ds = PanNukeTissueDataset(
        str(DATASET_TISSUES / tissue),
        split=split,
        image_transform=None,
        target_transform=None,
    )
    return DataLoader(ds, batch_size=BATCH_SIZE, shuffle=False, num_workers=2)

# Small EDA count table
records = []
for t in sorted(available_tissues):
    for split in ["train", "val", "test"]:
        try:
            ds = PanNukeTissueDataset(str(DATASET_TISSUES / t), split=split)
            records.append({"tissue": t, "split": split, "n": len(ds)})
        except Exception:
            pass
eda_df = pd.DataFrame(records).pivot(index="tissue", columns="split", values="n").fillna(0).astype(int)
eda_df.head()


In [ ]:
# U-Net baseline loader (checkpoint optional)

def load_unet_checkpoint(ckpt_path: Path, num_classes: int = 7) -> nn.Module:
    model = UNet(in_channels=3, num_classes=num_classes)
    if ckpt_path.exists():
        state = torch.load(ckpt_path, map_location=device)
        # allow raw state or dict
        state_dict = state.get('model_state', state)
        model.load_state_dict(state_dict, strict=False)
        print(f"Loaded U-Net weights from {ckpt_path}")
    else:
        print(f"U-Net checkpoint not found at {ckpt_path}; using randomly initialized model")
    model.to(device).eval()
    return model

# SAM wrapper (automatic mask generation -> instance map)
try:
    from segment_anything import sam_model_registry, SamAutomaticMaskGenerator
    SAM_AVAILABLE = True
except Exception as e:
    print("SAM not available:", e)
    SAM_AVAILABLE = False

class SAMWrapper:
    def __init__(self, model_type: str, checkpoint: str | None = None):
        assert SAM_AVAILABLE, "segment_anything not installed"
        if checkpoint and os.path.isfile(checkpoint):
            self.sam = sam_model_registry[model_type](checkpoint=checkpoint)
        else:
            self.sam = sam_model_registry[model_type]()
        self.mask_gen = SamAutomaticMaskGenerator(
            model=self.sam,
            points_per_side=32,
            pred_iou_thresh=0.7,
            stability_score_thresh=0.92,
            crop_n_layers=1,
            crop_n_points_downscale_factor=2,
            min_mask_region_area=80,
        )
    @torch.no_grad()
    def predict_instances(self, image_np: np.ndarray) -> np.ndarray:
        # image_np: HxWx3 uint8
        masks = self.mask_gen.generate(image_np)
        if not masks:
            return np.zeros(image_np.shape[:2], dtype=np.int32)
        inst = np.zeros(image_np.shape[:2], dtype=np.int32)
        for idx, m in enumerate(masks, start=1):
            inst[m['segmentation'].astype(bool)] = idx
        return inst


In [ ]:
# Inference helpers
from PIL import Image

@torch.no_grad()
def unet_predict_instances(model: nn.Module, img_tensor: torch.Tensor) -> np.ndarray:
    # img_tensor: 3xHxW (normalized)
    model.eval()
    logits = model(img_tensor.unsqueeze(0).to(device))
    sem = torch.argmax(logits, dim=1).squeeze(0).detach().cpu().numpy().astype(np.uint8)
    # derive boundary from semantic changes
    pad = np.pad(sem, 1, mode='edge')
    boundary = (sem != pad[1:-1, 1:-1]).astype(np.uint8)
    inst = reconstruct_instances(sem, boundary)
    return inst


def tensor_to_uint8(rgb_tensor: torch.Tensor) -> np.ndarray:
    # approximate inverse of default normalization for visualization/SAM
    arr = rgb_tensor.permute(1,2,0).cpu().numpy()
    arr = arr * np.array([0.229, 0.224, 0.225])[None,None,:] + np.array([0.485, 0.456, 0.406])[None,None,:]
    arr = np.clip(arr, 0, 1)
    return (arr * 255).astype(np.uint8)


def evaluate_on_tissue(tissue: str, models: Dict[str, object], n_limit: int | None = None) -> List[Dict]:
    loader = make_loader(tissue, split="test")
    results = []
    seen = 0
    for batch_idx, batch in enumerate(loader):
        images, targets = batch  # targets are semantic gt
        for b in range(images.shape[0]):
            if n_limit is not None and seen >= n_limit:
                return results
            img_t = images[b]
            gt_sem = targets[b].numpy()
            # GT instance reconstruction from sem + boundary
            pad = np.pad(gt_sem, 1, mode='edge')
            gt_boundary = (gt_sem != pad[1:-1, 1:-1]).astype(np.uint8)
            gt_inst = reconstruct_instances(gt_sem, gt_boundary)
            # Per-image id for pairing
            image_id = f"{tissue}/test/{batch_idx:05d}_{b}"
            # Evaluate each model (generic: if wrapper exposes predict_instances, use it)
            for name, model in models.items():
                if hasattr(model, 'predict_instances'):
                    img_u8 = tensor_to_uint8(img_t)
                    pred_inst = model.predict_instances(img_u8)
                else:
                    pred_inst = unet_predict_instances(model, img_t)
                # Metrics
                pq = pq_panoptic(gt_inst, pred_inst)
                f1o = f1_object(gt_inst, pred_inst)
                aji = aji_aggregated_jaccard(gt_inst, pred_inst)
                dice = dice_coefficient(gt_sem, (pred_inst > 0).astype(np.uint8), num_classes=2, ignore_background=False)
                results.append({
                    "tissue": tissue,
                    "image_id": image_id,
                    "model": name,
                    "pq": pq,
                    "f1_object": f1o,
                    "aji": aji,
                    "dice_bin": dice,
                })
            seen += 1
    return results


In [ ]:
# Configure models (gate by availability)
MODELS: Dict[str, object] = {}

# U-Net baseline checkpoint (update if you have a trained model)
UNET_CKPT = PROJECT_ROOT / "artifacts" / "rq3_enhanced" / "checkpoints" / "unet_original_enhanced_best.pth"
MODELS["unet_baseline"] = load_unet_checkpoint(UNET_CKPT, num_classes=7)

# SAM variants (require segment_anything + optional checkpoints)
if SAM_AVAILABLE:
    try:
        MODELS["sam_vit_b"] = SAMWrapper("vit_b")
    except Exception as e:
        print("Skipping sam_vit_b:", e)
    try:
        MODELS["sam_vit_l"] = SAMWrapper("vit_l")
    except Exception as e:
        print("Skipping sam_vit_l:", e)
    try:
        MODELS["sam_vit_h"] = SAMWrapper("vit_h")
    except Exception as e:
        print("Skipping sam_vit_h:", e)

print("Models configured:", list(MODELS.keys()))


In [ ]:
# TIAToolbox HoVer-Net integration (official)
try:
    from tiatoolbox.models.architecture.hovernet import HoVerNet
    from tiatoolbox.models.engine.semantic_segmentor import SemanticSegmentor
    from tiatoolbox.utils.transforms import imresize
    TIA_AVAILABLE = True
except Exception as e:
    print("TIAToolbox not available:", e)
    TIA_AVAILABLE = False

class TIAHoverNetWrapper:
    def __init__(self, weights_path: str | None = None, pretrained: str | None = 'hovernet_fast-pannuke'):
        assert TIA_AVAILABLE, "TIAToolbox is required for HoVer-Net integration"
        # SemanticSegmentor wraps HoVerNet with official postprocessing
        self.segmentor = SemanticSegmentor(backbone='hovernet', pretrained=pretrained)
        if weights_path and os.path.isfile(weights_path):
            self.segmentor.model.load_state_dict(torch.load(weights_path, map_location='cpu'), strict=False)
        self.segmentor.model.to(device)
        self.segmentor.model.eval()

    @torch.no_grad()
    def predict_instances(self, image_np: np.ndarray) -> np.ndarray:
        # TIAToolbox expects uint8 RGB; returns dict with inst_map for PanNuke models
        # resize to 256 if needed to match training size
        out = self.segmentor.predict(np.expand_dims(image_np, 0))
        # out is list of dicts; prefer 'inst_map' when available
        if isinstance(out, list) and len(out) > 0:
            pred = out[0]
            if isinstance(pred, dict) and 'inst_map' in pred:
                return pred['inst_map'].astype(np.int32)
        # Fallback: no instances
        return np.zeros(image_np.shape[:2], dtype=np.int32)

# Add TIAToolbox HoVer-Net (if available)
try:
    if TIA_AVAILABLE:
        MODELS['hovernet_tia'] = TIAHoverNetWrapper(pretrained='hovernet_fast-pannuke')
        print('Added TIAToolbox HoVer-Net (pretrained: hovernet_fast-pannuke)')
except Exception as e:
    print('Skipping TIAToolbox HoVer-Net:', e)


In [ ]:
# HoVer-Net integration (inline lightweight implementation)
class HoverNet(nn.Module):
    def __init__(self, num_classes: int = 7, backbone: str = 'resnet34'):
        super().__init__()
        import torchvision.models as tvm
        if backbone == 'resnet50':
            m = tvm.resnet50(weights=tvm.ResNet50_Weights.DEFAULT)
            enc_channels = 2048
        else:
            m = tvm.resnet34(weights=tvm.ResNet34_Weights.DEFAULT)
            enc_channels = 512
        self.encoder = nn.Sequential(*list(m.children())[:-2])
        self.up1 = nn.ConvTranspose2d(enc_channels, 256, 2, 2)
        self.dec1 = nn.Sequential(nn.Conv2d(256, 256, 3, padding=1), nn.ReLU(True))
        self.up2 = nn.ConvTranspose2d(256, 128, 2, 2)
        self.dec2 = nn.Sequential(nn.Conv2d(128, 128, 3, padding=1), nn.ReLU(True))
        self.up3 = nn.ConvTranspose2d(128, 64, 2, 2)
        self.dec3 = nn.Sequential(nn.Conv2d(64, 64, 3, padding=1), nn.ReLU(True))
        self.up4 = nn.ConvTranspose2d(64, 64, 2, 2)
        self.classifier = nn.Conv2d(64, num_classes, 1)
    def forward(self, x: torch.Tensor) -> torch.Tensor:
        f = self.encoder(x)
        x = self.up1(f)
        x = self.dec1(x)
        x = self.up2(x)
        x = self.dec2(x)
        x = self.up3(x)
        x = self.dec3(x)
        x = self.up4(x)
        return self.classifier(x)

def load_hovernet_checkpoint(ckpt_path: Path, num_classes: int = 7) -> nn.Module:
    model = HoverNet(num_classes=num_classes)
    if ckpt_path.exists():
        state = torch.load(ckpt_path, map_location=device)
        if isinstance(state, dict) and any(k.startswith('model') for k in state.keys()):
            state = state.get('model_state', state.get('model_state_dict', state))
        model.load_state_dict(state, strict=False)
        print(f'Loaded HoVer-Net weights from {ckpt_path}')
    else:
        print(f'HoVer-Net checkpoint not found at {ckpt_path}; using randomly initialized model')
    model.to(device).eval()
    return model

# Add HoVer-Net
HOVERNET_CKPT = PROJECT_ROOT / 'artifacts' / 'rq1' / 'checkpoints' / 'hovernet_best.pth'
try:
    MODELS['hovernet'] = load_hovernet_checkpoint(HOVERNET_CKPT, num_classes=7)
except Exception as e:
    print('Skipping HoVer-Net:', e)

print('Models configured (after HoVer-Net attempt):', list(MODELS.keys()))


In [ ]:
# Run evaluation across tissues (limit per-tissue for quick pass)
ALL_ROWS: List[Dict] = []
per_tissue_limit = 50  # increase for full evaluation

for tissue in sorted(available_tissues):
    print(f"Evaluating {tissue} ...")
    rows = evaluate_on_tissue(tissue, MODELS, n_limit=per_tissue_limit)
    ALL_ROWS.extend(rows)

res_df = pd.DataFrame(ALL_ROWS)
print(res_df.head())

# Save per-image table
csv_path = CSV_DIR / "per_image_instance_metrics.csv"
res_df.to_csv(csv_path, index=False)
print("Saved:", csv_path)


In [ ]:
# Summary stats per model
summary = (
    res_df.groupby("model")["pq", "f1_object", "aji", "dice_bin"].agg(["mean", "std", "count"]).round(4)
)
summary


In [ ]:
# Pairwise statistical analysis with BH correction
from itertools import combinations
from statsmodels.stats.multitest import multipletests
from scipy.stats import ttest_rel, wilcoxon

metrics = ["pq", "f1_object", "aji", "dice_bin"]
models = res_df["model"].unique().tolist()

pairwise_rows = []
for m1, m2 in combinations(models, 2):
    df1 = res_df[res_df.model == m1].set_index(["tissue", "image_id"])  # align by image
    df2 = res_df[res_df.model == m2].set_index(["tissue", "image_id"])  # same ids
    common_idx = df1.index.intersection(df2.index)
    if len(common_idx) < 5:
        continue
    for metric in metrics:
        x = df1.loc[common_idx, metric].values
        y = df2.loc[common_idx, metric].values
        if len(x) != len(y) or len(x) < 5:
            continue
        # Paired tests
        t_stat, t_p = ttest_rel(x, y, nan_policy='omit')
        try:
            w_stat, w_p = wilcoxon(x, y)
        except Exception:
            w_stat, w_p = np.nan, np.nan
        diff = np.nanmean(x - y)
        pairwise_rows.append({
            "model1": m1,
            "model2": m2,
            "metric": metric,
            "n": int(len(x)),
            "mean_diff": float(diff),
            "t_p": float(t_p) if np.isfinite(t_p) else 1.0,
            "w_p": float(w_p) if np.isfinite(w_p) else 1.0,
        })

pairwise_df = pd.DataFrame(pairwise_rows)
if not pairwise_df.empty:
    # BH correction per metric separately
    corrected = []
    for metric, g in pairwise_df.groupby("metric"):
        for col in ["t_p", "w_p"]:
            rej, p_bh, _, _ = multipletests(g[col].values, method='fdr_bh')
            g[col+"_bh"] = p_bh
            g[col+"_sig_bh"] = rej
        corrected.append(g)
    pairwise_df = pd.concat(corrected, ignore_index=True)

pairwise_csv = CSV_DIR / "pairwise_stats_bh.csv"
pairwise_df.to_csv(pairwise_csv, index=False)
print("Saved:", pairwise_csv)

pairwise_df.head()


In [ ]:
# Plots
plt.figure(figsize=(8,4))
sns.boxplot(data=res_df, x="model", y="pq")
plt.xticks(rotation=30, ha='right')
plt.title("PQ by model")
plt.tight_layout()
fig_path1 = FIG_DIR / "pq_by_model.png"
plt.savefig(fig_path1, dpi=200)
plt.close()

plt.figure(figsize=(8,4))
sns.boxplot(data=res_df, x="model", y="f1_object")
plt.xticks(rotation=30, ha='right')
plt.title("Object F1 by model")
plt.tight_layout()
fig_path2 = FIG_DIR / "f1_by_model.png"
plt.savefig(fig_path2, dpi=200)
plt.close()

print("Saved:", fig_path1)
print("Saved:", fig_path2)


In [ ]:
# Per-tissue paired Wilcoxon tests (BH corrected)
from itertools import product

metrics_primary = ["pq", "f1_object"]
sam_models = [m for m in res_df.model.unique() if m.startswith("sam")]
established = [m for m in ["hovernet", "cellvit", "lkcell"] if m in res_df.model.unique()]

rows = []
for tissue in sorted(res_df.tissue.unique()):
    df_t = res_df[res_df.tissue == tissue].set_index(["tissue", "image_id"])  # align pairs
    for sam, est, metric in product(sam_models, established, metrics_primary):
        a = df_t[df_t.model == sam][metric]
        b = df_t[df_t.model == est][metric]
        idx = a.index.intersection(b.index)
        if len(idx) < 5:
            continue
        x, y = a.loc[idx].values, b.loc[idx].values
        try:
            stat, p = wilcoxon(x, y)
        except Exception:
            p = 1.0
        rows.append({
            "tissue": tissue,
            "sam": sam,
            "established": est,
            "metric": metric,
            "n": int(len(idx)),
            "mean_diff": float(np.nanmean(x - y)),
            "wilcoxon_p": float(p)
        })

tissue_df = pd.DataFrame(rows)
if not tissue_df.empty:
    outs = []
    for metric, g in tissue_df.groupby("metric"):
        rej, p_bh, _, _ = multipletests(g["wilcoxon_p"].values, method="fdr_bh")
        g = g.assign(wilcoxon_p_bh=p_bh, sig_bh=rej)
        outs.append(g)
    tissue_df_bh = pd.concat(outs, ignore_index=True)
else:
    tissue_df_bh = pd.DataFrame(columns=["tissue","sam","established","metric","n","mean_diff","wilcoxon_p","wilcoxon_p_bh","sig_bh"])

per_tissue_csv = CSV_DIR / "per_tissue_wilcoxon_bh.csv"
tissue_df_bh.to_csv(per_tissue_csv, index=False)
print("Saved:", per_tissue_csv)

tissue_df_bh.head()


In [ ]:
# HTML report (with per-tissue section)
from datetime import datetime

report_html = f"""
<!DOCTYPE html>
<html><head><meta charset='utf-8'><title>RQ1 - SAM Variants vs Baselines</title></head>
<body style='font-family:Segoe UI,Arial,sans-serif; margin:40px;'>
<h1>RQ1: SAM Variants vs Established Models on PanNuke</h1>
<p><em>Generated: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}</em></p>
<h2>Per-image metrics</h2>
<p>Saved CSV: {csv_path.name}</p>
<h2>Summary (by model)</h2>
{summary.to_html()}
<h2>Pairwise statistics (BH corrected)</h2>
{pairwise_df.head(50).to_html(index=False) if 'pairwise_df' in globals() and not pairwise_df.empty else '<p>No pairwise results.</p>'}
<h2>Per-tissue paired Wilcoxon (BH corrected)</h2>
{tissue_df_bh.head(100).to_html(index=False) if 'tissue_df_bh' in globals() and not tissue_df_bh.empty else '<p>No per-tissue results.</p>'}
<h2>Figures</h2>
<ul>
  <li>{fig_path1.name}</li>
  <li>{fig_path2.name}</li>
</ul>
</body></html>
"""
html_path = REPORTS_DIR / "RQ1_SAM_Variants_Report.html"
html_path.write_text(report_html, encoding='utf-8')
print("Saved:", html_path)


In [ ]:
# HTML report
from datetime import datetime

report_html = f"""
<!DOCTYPE html>
<html><head><meta charset='utf-8'><title>RQ1 - SAM Variants vs Baselines</title></head>
<body style='font-family:Segoe UI,Arial,sans-serif; margin:40px;'>
<h1>RQ1: SAM Variants vs Established Models on PanNuke</h1>
<p><em>Generated: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}</em></p>
<h2>Per-image metrics</h2>
<p>Saved CSV: {csv_path.name}</p>
<h2>Summary (by model)</h2>
{summary.to_html()}
<h2>Pairwise statistics (BH corrected)</h2>
{pairwise_df.head(50).to_html(index=False)}
<h2>Figures</h2>
<ul>
  <li>{fig_path1.name}</li>
  <li>{fig_path2.name}</li>
</ul>
</body></html>
"""
html_path = REPORTS_DIR / "RQ1_SAM_Variants_Report.html"
html_path.write_text(report_html, encoding='utf-8')
print("Saved:", html_path)
